# Correlate labeled data to checkout data

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import math
import csv
import time
from datetime import datetime, timedelta, tzinfo
import numpy as np
import pandas as pd
import pprint
#import matching as mt

current_item = 'Bier'

### Read groundtruth - Merge categories

In [ ]:
label_names = {}
path = '/nfs/students/winter-term-2018/project_2/test/ids_to_name.csv'
with open(path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        label_names[row[1]] = row[2]
df = pd.read_csv(path)
df.head(len(label_names))

In [ ]:
merged_categories = {}
category_names = ['Bier', 'Bier Maß', 'Weißbier', 'Cola', 'Wasser', 'Curry-Wurst', 'Weißwein',
                   'A-Schorle', 'Jägermeister', 'Pommes', 'Burger', 'Williamsbirne', 'Alm-Breze', 'Brotzeitkorb',
                   'Käsespätzle']
merged_categories['100'] = 0
merged_categories['101'] = 1
merged_categories['102'] = 0
merged_categories['103'] = 1
merged_categories['104'] = 0
merged_categories['105'] = 1
merged_categories['106'] = 2
merged_categories['107'] = 1
merged_categories['108'] = 2
merged_categories['109'] = 1
merged_categories['110'] = 2
merged_categories['111'] = 1
#merged_categories['112'] = None #Mass AFG
merged_categories['113'] = 3
merged_categories['114'] = 3
merged_categories['115'] = 3
merged_categories['116'] = 3
merged_categories['117'] = 4
merged_categories['118'] = 7
#merged_categories['119'] = None #RedBull
merged_categories['121'] = 4
merged_categories['122'] = 4
#merged_categories['123'] = None #Kaffee
#merged_categories['124'] = None #Cappuccino
#merged_categories['125'] = None #Secco Rose
#merged_categories['126'] = None
merged_categories['128'] = 4
merged_categories['129'] = 6
merged_categories['130'] = 6
merged_categories['131'] = 6
#merged_categories['132'] = None #lugana flasche
#merged_categories['133'] = None #Grauer Burgunder
merged_categories['134'] = 3
merged_categories['173'] = 8
merged_categories['197'] = 5
merged_categories['204'] = 9
merged_categories['206'] = 9
merged_categories['207'] = 9
merged_categories['196'] = 10
merged_categories['171'] = 11
merged_categories['201'] = 12
merged_categories['199'] = 13
merged_categories['205'] = 14


for i in range(len(category_names)):
    print("Merged in category %s (%d):"%(category_names[i], i))
    for key in merged_categories:
        if merged_categories[key] == i:
            print("\t %s"%label_names[key])

In [ ]:
checkout_logs = {}
test_d = {}
with open('/nfs/students/winter-term-2018/project_2/labels/schanzerAlmData.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(row)
            line_count += 1
        else:
            if row[4] in merged_categories:
                time = datetime.strptime(row[1]+' '+row[2], '%Y-%m-%d %H:%M:%S')
                if time in checkout_logs:
                    new_item = [int(merged_categories[row[4]]), float(row[3])]
                    was_in_list = False
                    for i in range(len(checkout_logs[time])):
                        if new_item[0] == checkout_logs[time][i][0]:
                            checkout_logs[time][i][1] += new_item[1]
                            was_in_list = True
                    if not was_in_list:
                        checkout_logs[time].append(new_item)
                else:
                    checkout_logs[time] = [[int(merged_categories[row[4]]), float(row[3])]]
                if merged_categories[row[4]] in test_d:
                    test_d[merged_categories[row[4]]] += int(float(row[3]))
                else:
                    test_d[merged_categories[row[4]]] = int(float(row[3]))
            line_count += 1
    print(f'Processed {line_count} lines.')
#checkout_logs.sort(key=lambda first: first[0])
#print("First checkout: %s"%str(checkout_logs[0]))
#print("Last checkout: %s"%str(checkout_logs[-1]))
print(test_d)
#pprint.pprint(checkout_logs)

def checkout_log_to_name(checkout_log):
    data = checkout_logs[checkout_log]
    s = ''
    for d in data:
        #print(d)
        s += str(d[1])+'x '+str(category_names[d[0]])+' '
    return s

In [ ]:
plt.bar(test_d.keys(), test_d.values(), width=1.0)
plt.show()
max_item = max(test_d, key=test_d.get)
min_item = min(test_d, key=test_d.get)
print("Most sold item was %s sold %s time(s) "%(category_names[max_item], test_d[max_item]))
print("Fewest sold item was %s sold %s time(s) "%(category_names[min_item], test_d[min_item]))

### Correlate labeled data to checkouts

In [ ]:
def ts_to_time(ts, tz=None):
    #return datetime.utcfromtimestamp(int(ts))
    return datetime.fromtimestamp(int(ts), tz=tz)
print(ts_to_time('1526731043'))

In [ ]:
"""
Read labels from file
"""
label_file = open('/nfs/students/winter-term-2018/project_2/data_split/train/thumbnails/'+current_item+'/files.txt', mode='r').read()
annotations = label_file.split('\n')
annotations.remove('')
labeled_image_dict = {}
image_data_dict = {}
for a in annotations:
    items = a.split(' ')
    try:
        if items[1] is '0':
            continue
    except:
        print(items)
        continue
    labels = []
    for i in range(int(items[1])):
        labels.append(items[5*i+2:5*i+7])
        labels[i][0] = int(labels[i][0])
    time = ts_to_time(items[0][:10])
    if time in labeled_image_dict:
        if int(items[1]) > labeled_image_dict[time][0]:
            print("Found label of timeframe with more labels:\nOld: %s\nNew: %s"%(labeled_image_dict[time], (int(items[1]),labels)))    
            labeled_image_dict[time] = (int(items[1]), labels)
            image_data_dict[time] = plt.imread('/nfs/students/winter-term-2018/project_2/data_split/train/thumbnails/'+current_item+'/'+items[0])
    else:
        #add_time = timedelta(seconds=float(items[0][14:16])*6.0/90.0)
        #print(add_time)
        labeled_image_dict[time] = (int(items[1]), labels)
        image_data_dict[time] = plt.imread('/nfs/students/winter-term-2018/project_2/data_split/train/thumbnails/'+current_item+'/'+items[0])
        


In [ ]:
#Uncomment for detailed checkout logs
#pprint.pprint(checkout_logs)

In [ ]:
test_img = annotations[0].split(' ')[0]
print(test_img)
test_date = ts_to_time(test_img[:10])
print(test_date.astimezone())
print(test_date)
path = '/nfs/students/winter-term-2018/project_2/data_split/train/thumbnails/'+current_item+'/'+test_img
img_data = plt.imread(path)
if not img_data is None:
    plt.imshow(img_data[:40,:330])
    plt.show()
else:
    print("Error")

### Create Matching problem

In [ ]:
"""
First step: filter 
Remark: Very naively implemented (could use LSH filtering or stuff to make it faster)
"""
max_delta = timedelta(minutes=1)
print("Maximal timedelta from checkout to image is %s"%str(max_delta))
count = 0
count_nf = 0
print_stuff = False
filter_assignment = {}
not_found_examples = []
for image in labeled_image_dict:
    found = False
    for checkout in checkout_logs:
        dif = abs(checkout - image)
        if abs(dif.days) <= max_delta.days and dif < max_delta:#neccessary because of overflows
            if print_stuff and count < 20:
                print("Checkout: %s\tImage: %s\tDifference %s"%(checkout, image, str(checkout-image)))
            if image in filter_assignment:
                filter_assignment[image].append(checkout)
            else:
                filter_assignment[image] = [checkout]
            count += 1
            found = True
        i,c = image, checkout
    if not found:
        not_found_examples.append(image)
        count_nf += 1
        
print("Did not found checkouts within delta t for %d/%d images"%(count_nf, len(labeled_image_dict)))
print("Found %d candidate pairs"%count)

In [ ]:
def calc_similarity(image, checkout):
    image_data = labeled_image_dict[image]
    checkout_data = checkout_logs[checkout]
    #print(image_data)
    #print(checkout_data)
    categories = {}
    for obj in image_data[1]:
        if obj[0] in categories:
            categories[obj[0]] = [0.0, categories[obj[0]][1]+1.0]
        else:
            categories[obj[0]] = [0.0, 1.0]
    for item in checkout_data:
        if item[0] in categories:
            categories[item[0]] = [item[1], categories[item[0]][1]]
        else:
            categories[item[0]] = [item[1], 0]
    #print(categories)
    score = 0.0
    count = 0.0
    for item, amt in categories.items():
        score += max(amt)-abs(amt[0]-amt[1])#how many matching data points
        count += max(amt)                   #how many data points in total
    #print("Score = %f\tCount = %f"%(score, count))
    score /= count
    return score

In [ ]:
print(calc_similarity(i, c))

In [ ]:
"""
Second step: evaluate simlilarity of candidate pairs
i.e. is order and label identical (or at least similar)
"""
count = 0
print_stuff = False
time_difference_matches = []
image_checkout_matches = []
score_series = []
threshold = 0.5
for image, possible_matches in filter_assignment.items():
    #pprint.pprint("Image %s has %d possible match(es) %s"%(image,len(possible_matches), possible_matches))
    ranking = []
    best_sim_score = 0.0
    best_match = None
    for match in possible_matches:
        sim_score = calc_similarity(image, match)
        if sim_score > best_sim_score:
            best_sim_score = sim_score
            best_match = match
        if sim_score > threshold:
            ranking.append((match, sim_score))
            #print("Image %s and checkout %s have score of %f"%(labeled_image_dict[image], checkout_logs[match],sim_score))
    if len(ranking) == 0:
        count += 1
        if print_stuff:
            print("Could not match labels of %s to following checkouts:"%str(labeled_image_dict[image]))
            for m in possible_matches:
                print("\t%s"%str(checkout_logs[m]))
    else:
        time_difference_matches.append(abs(image-best_match).total_seconds())
        image_checkout_matches.append((image, best_match))
    if print_stuff:
        print("Best similarity score is %f"%best_sim_score)
    score_series.append(best_sim_score)
print("No good correspondances for %d/%d images"%(count, len(filter_assignment)))

In [ ]:
df = pd.Series(time_difference_matches)
df.describe()

In [ ]:
df_bs = pd.Series(score_series)
df_bs.describe()

In [ ]:
for img, checkout in image_checkout_matches:
    #print("Image %s matched to checkout %s"%(img, checkout))
    plt.imshow(image_data_dict[img])
    plt.title(checkout_log_to_name(checkout))
    plt.show()

<b>Scatter plot of assignments</b>

In [ ]:
x_p = []
y_p = []
day = 21
min_h = 12
max_h = 13
for image in labeled_image_dict:
    if image.day == day and image.hour > min_h and image.hour <= max_h:
        x_p.append(image.timestamp())
        y_p.append(1.0)
for checkout in checkout_logs:
    if checkout.day == day and checkout.hour > min_h and checkout.hour <= max_h:
        x_p.append(checkout.timestamp())
        y_p.append(0.0)

In [ ]:
plt.scatter(x_p, y_p)
for match in image_checkout_matches:
    plt.arrow(match[0].timestamp(), 1.0, match[1].timestamp()-match[0].timestamp(), -1.0)
plt.yticks([0.0,1.0],['Checkouts','Images'])
plt.xticks([])
plt.show()

In [ ]:
for example in not_found_examples:
    checkouts = []
    x = []
    td = timedelta(minutes=30)
    for checkout in checkout_logs:
        if abs(checkout-example) < td:
            checkouts.append(checkout)
            x.append(checkout.timestamp())
    y = [0.0] * len(x)
    x.append(example.timestamp())
    y.append(1.0)
    print(x)
    print(y)
    plt.scatter(x,y)
    plt.arrow((example+timedelta(minutes=5)).timestamp(), 0.0, 0.0, 1.0)
    plt.arrow((example-timedelta(minutes=5)).timestamp(), 0.0, 0.0, 1.0)
    plt.arrow((example-timedelta(minutes=5)).timestamp(), 0.5, timedelta(minutes=10).total_seconds(), 0.0, label='10 minutes')
    plt.text((example-timedelta(minutes=5)).timestamp(), 0.55, '10 minutes')
    plt.yticks([0.0,1.0],['Checkouts','Images'])
    plt.xticks([])
    plt.show()
    

In [ ]:
len(not_found_examples)